In [ ]:
#imports
import pandas as pd
import scipy 
from scipy import stats
import sklearn.preprocessing
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Collection:

In [ ]:
#imported xlsx files, downloaded from sources

# Preprocessing:

In [ ]:
#PLAN - find yearly avg global temp, add to each entry according to year, find avg estimate for each unique indicator_name in each year, use the appropriate indicator_names to form a score for inequality and train model on x - temp, year | y-inequality scale

df = pd.read_csv('GlobalLandTemperaturesByCountry.csv')
df = df.dropna()
df = df.drop(columns='AverageTemperatureUncertainty')
#dt column -> date: yy/mm/dd
#get an average temp for each year
df['dt'] = pd.to_datetime(df['dt'])
df['year'] = df['dt'].dt.year
yearly_avg_temp = df.groupby('year')['AverageTemperature'].mean().reset_index()

yearly_temps = yearly_avg_temp.set_index('year')['AverageTemperature'].to_numpy()
unique_years_enviro = df['year'].unique()
#to get yearly avg temp for a year -> yearly_temps[desired year - 1743]

df = pd.read_excel('enviro_data.xlsx')
print(df.head())

In [ ]:
#target -> health inequality 
#POA -> for each year, find the average estimate for each indicator_name type
# -> then append a yearly_avg_temp column that will contain the avg temp for the year fo the entry
df = df.groupby(['date', 'indicator_name'])['estimate'].mean().reset_index()
print(df.info())
indicator_names = df['indicator_name'].unique()
file = open('unique_values.txt', 'w')
file.write(str(indicator_names))
file.close()

file = open('unique_years.txt', 'w')
file.write(str(unique_years_enviro))
file.close()
#relevant columns from the WHO data -> 'date', 'indicator_name', 'subgroup' 

In [ ]:
df = df[df['date'].isin(unique_years_enviro)]
df = df.iloc[:281] #only have temp up to 2013
avg_temp_year = []
for row in df.itertuples():
    avg_temp_year.append(yearly_temps[row.date - 1784])

df['avg_yearly_temp'] = avg_temp_year
print(df.head())

In [ ]:
#noramlize between 0 and 1
df['estimate'] = (df['estimate'] - df['estimate'].min())/(df['estimate'].max() - df['estimate'].min())

print(df.head())

In [ ]:
yearly_health_ineq_factors = []
estimates = []
start_year = 1990
unique_years_df = [1990]
for row in df.itertuples():
    if row.date == start_year:
        estimates.append(row.estimate)
    else:
        total = sum(estimates)
        total *= (1/len(estimates))
        yearly_health_ineq_factors.append(total)
        estimates = []
        estimates.append(row.estimate)
        start_year += 1
        unique_years_df.append(start_year)
        
estimates = []
for row in df.itertuples():
    if row.date == 2013:
        estimates.append(row.estimate)
        
total = sum(estimates)
total *= (1/len(estimates))
yearly_health_ineq_factors.append(total)

print(unique_years_df)
#put on a scale of 10
stand_factors = [10 * (value - min(yearly_health_ineq_factors)) / (max(yearly_health_ineq_factors) - min(yearly_health_ineq_factors)) for value in yearly_health_ineq_factors]

In [ ]:
#standardize temp
stand_temps = yearly_temps[-24:]
stand_temps = [(value - min(yearly_temps))/(max(yearly_temps) - min(yearly_temps)) for value in stand_temps]

In [ ]:
df = pd.DataFrame({ 'year': list(range(1990, 2014)), 'health_ineq_factor':stand_factors, 'stand_temp':stand_temps, 'temp':yearly_temps[-24:]})
print(df)

In [ ]:
#Train linear regression model
X = df['stand_temp']
y = df['health_ineq_factor']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(y_train)
X_test = pd.DataFrame(X_test)
y_test = pd.DataFrame(y_test)
linearRegression = LinearRegression(fit_intercept=True)
linearRegression.fit(X_train, y_train)



In [ ]:
y_prediction = linearRegression.predict(X_test)
print(y_prediction)
mse = mean_squared_error(y_test, y_prediction)
r2 = r2_score(y_test, y_prediction)

print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {np.sqrt(mse)}")
print(f"R-squared: {r2}")

In [ ]:
malaria = pd.read_csv('global-malaria.csv')
malaria.drop([1, 54, 55, 65, 66, 104, 123, 140, 160, 188, 219, 221, 222, 223, 224, 220], inplace=True)
malaria.drop('Region', axis=1, inplace=True)
malaria.info()

In [ ]:
y = pd.DataFrame(malaria.std())
scaler = StandardScaler()

y=pd.DataFrame(scaler.fit_transform(y))
y.describe()
df['health_ineq_factor'] = y[0]
df

In [ ]:
X = df['stand_temp']
y = df['health_ineq_factor']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = pd.DataFrame(X_train)
y_train = pd.DataFrame(y_train)
X_test = pd.DataFrame(X_test)
y_test = pd.DataFrame(y_test)
linearRegression = LinearRegression(fit_intercept=True)
linearRegression.fit(X_train, y_train)



In [ ]:
y_prediction = linearRegression.predict(X_test)
print(y_prediction)
mse = mean_squared_error(y_test, y_prediction)
r2 = r2_score(y_test, y_prediction)

print(f"Mean Squared Error: {mse}")
print(f"Root Mean Squared Error: {np.sqrt(mse)}")
print(f"R-squared: {r2}")